# **0. Load libraries**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#!pip install implicit
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from implicit.als import AlternatingLeastSquares


In [ ]:
# og_file_path="~/code/Alanoudis/food-delivery-rec/data/updated_data/orders_sg25k.txt"
# og_data = pd.read_csv(og_file_path, sep=',', encoding='utf-8')
# og_data.head()

# og_products = "~/code/Alanoudis/food-delivery-rec/data/raw_data/products_sg.txt"
# og_products = pd.read_csv(og_products, sep=',', encoding='utf-8')
# og_products.head()

# og_vendors = "~/code/Alanoudis/food-delivery-rec/data/raw_data/vendors_sg.txt"
# og_vendors = pd.read_csv(og_vendors, sep=',', encoding='utf-8')
# og_vendors.head()

,Unnamed: 0,vendor_id,chain_id,geohash,primary_cuisine
0,0,b160c319,d2786168,w21z6,mexican
1,1,9c8f010e,d2786168,w21ze,mexican
2,4,03eb25e1,5055ab25,w21ze,bak kut teh
3,5,3613129a,8984acb6,w23b1,italian
4,7,0946c9e5,1e3a2913,w21z4,bak kut teh


In [3]:
full_data_path = "~/code/Alanoudis/food-delivery-rec/notebooks/alanoud/full_data.txt"
full_data = pd.read_csv(full_data_path, sep=',', encoding='utf-8')
full_data.head()

,customer_id,customer_geohash,order_id,vendor_id,day_of_week,order_time,order_day,chain_id,vendor_geohash,cuisine_origin,avg_vendor_rating,num_products,total_order_value,products_ordered
0,008ab40ac0,w21z7,9390,8ace9ccb,2,01:52:03,3 days,aece2f12,w21z7,chinese,3.5,1,0.4,Spicy中辣
1,008ce71183,w21zb,7057,f0d84faa,5,18:11:48,34 days,fc3b6153,w21zc,chinese,3.5,2,12.8,"Kway Teow Goreng with Petai & Prawns 虾仁臭豆炒河粉, ..."
2,008ce71183,w21zb,7058,a23e4559,2,19:22:26,31 days,788f82f6,w21zb,american,3.8,4,14.8,"Tuna D'Licious, Spicy Chicken Pizza Baguette, ..."
3,008ce71183,w21zb,7059,a23e4559,3,18:54:16,18 days,788f82f6,w21zb,american,3.6,4,11.6,"Tuna D'Licious, Chocolate Eclair, Tuna D'Licio..."
4,00ba08bab4,w21zt,347,78ce75cb,3,20:39:37,39 days,24975bf7,w21zt,american,4.5,1,3.6,McGriddles Feast


In [5]:
df = full_data.copy()
df.head()

,customer_id,customer_geohash,order_id,vendor_id,day_of_week,order_time,order_day,chain_id,vendor_geohash,cuisine_origin,avg_vendor_rating,num_products,total_order_value,products_ordered
0,008ab40ac0,w21z7,9390,8ace9ccb,2,01:52:03,3 days,aece2f12,w21z7,chinese,3.5,1,0.4,Spicy中辣
1,008ce71183,w21zb,7057,f0d84faa,5,18:11:48,34 days,fc3b6153,w21zc,chinese,3.5,2,12.8,"Kway Teow Goreng with Petai & Prawns 虾仁臭豆炒河粉, ..."
2,008ce71183,w21zb,7058,a23e4559,2,19:22:26,31 days,788f82f6,w21zb,american,3.8,4,14.8,"Tuna D'Licious, Spicy Chicken Pizza Baguette, ..."
3,008ce71183,w21zb,7059,a23e4559,3,18:54:16,18 days,788f82f6,w21zb,american,3.6,4,11.6,"Tuna D'Licious, Chocolate Eclair, Tuna D'Licio..."
4,00ba08bab4,w21zt,347,78ce75cb,3,20:39:37,39 days,24975bf7,w21zt,american,4.5,1,3.6,McGriddles Feast


# 1. Implicit Feedback Matrix Features:
Weighted scores combining order frequency and ratings (Use orders as strong preference signals).

User-Vendor matrix: Interaction strength between customers and vendors (Customer’s favorite restaurants based on order frequency)

User-Cuisine matrix: Preference strength for cuisine types (“Who likes which types of food?")

In [7]:
df.shape

(10581, 14)

In [12]:
# =============================================
# 1.1 IMPLICIT FEEDBACK MATRIX CONSTRUCTION
# =============================================

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from implicit.als import AlternatingLeastSquares
# Create implicit feedback matrices
def create_implicit_matrices(df):
    """
    Create User-Vendor and User-Cuisine interaction matrices
    """

    # User-Vendor Matrix (Order frequency + rating weighted)
    user_vendor_interactions = df.groupby(['customer_id', 'vendor_id']).agg({
        'num_products': 'sum',  # Using num_products as frequency proxy
        'avg_vendor_rating': 'mean',  # Using avg_vendor_rating instead of rating
        'order_id': 'count'
    }).reset_index()

    # Calculate weighted score (frequency * normalized rating)
    user_vendor_interactions['weighted_score'] = (
        user_vendor_interactions['order_id'] *
        (user_vendor_interactions['avg_vendor_rating'] / 5.0)  # Assuming 5-point scale
    )

    # Pivot to matrix format
    user_vendor_matrix = user_vendor_interactions.pivot_table(
        index='customer_id',
        columns='vendor_id',
        values='weighted_score',
        fill_value=0
    )

    # User-Cuisine Matrix
    user_cuisine_interactions = df.groupby(['customer_id', 'cuisine_origin']).agg({
        'num_products': 'sum',
        'avg_vendor_rating': 'mean',
        'order_id': 'count'
    }).reset_index()

    user_cuisine_interactions['weighted_score'] = (
        user_cuisine_interactions['order_id'] *
        (user_cuisine_interactions['avg_vendor_rating'] / 5.0)
    )

    user_cuisine_matrix = user_cuisine_interactions.pivot_table(
        index='customer_id',
        columns='cuisine_origin',
        values='weighted_score',
        fill_value=0
    )

    return user_vendor_matrix, user_cuisine_matrix, user_vendor_interactions, user_cuisine_interactions

# Create the matrices
user_vendor_matrix, user_cuisine_matrix, uv_interactions, uc_interactions = create_implicit_matrices(df)

print(f"User-Vendor Matrix: {user_vendor_matrix.shape}")
print(f"User-Cuisine Matrix: {user_cuisine_matrix.shape}")

User-Vendor Matrix: (2541, 3309)
User-Cuisine Matrix: (2541, 17)


In [14]:
# =============================================
# 1.2 PREFERENCE MODELING
# =============================================

# A) COLLABORATIVE FILTERING WITH ALS
def train_als_model(interaction_matrix, factors=50, iterations=15):
    """
    Train ALS model for collaborative filtering
    """
    # Convert to sparse matrix format
    sparse_matrix = csr_matrix(interaction_matrix.values)

    # Initialize and train ALS model
    als_model = AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        regularization=0.01,
        random_state=42
    )

    # Train the model
    als_model.fit(sparse_matrix)

    return als_model, sparse_matrix

# Train ALS on user-vendor interactions
als_model, sparse_user_vendor = train_als_model(user_vendor_matrix)

# Get user and item factors
user_factors = als_model.user_factors
vendor_factors = als_model.item_factors

print(f"User factors shape: {user_factors.shape}")
print(f"Vendor factors shape: {vendor_factors.shape}")

# B) CONTENT-BASED USER PROFILES
def create_content_based_profiles(df):
    """
    Create weighted user profiles based on cuisine preferences and price points
    """

    # 1. Cuisine Preference Profiles
    cuisine_preferences = df.groupby(['customer_id', 'cuisine_origin']).agg({
        'order_id': 'count',
        'avg_vendor_rating': 'mean',
        'total_order_value': 'mean'
    }).reset_index()

    # Calculate cuisine preference scores
    cuisine_preferences['preference_score'] = (
        cuisine_preferences['order_id'] *
        (cuisine_preferences['avg_vendor_rating'] / 5.0)
    )

    # Normalize scores per user
    user_total_scores = cuisine_preferences.groupby('customer_id')['preference_score'].transform('sum')
    cuisine_preferences['normalized_score'] = cuisine_preferences['preference_score'] / user_total_scores

    # Pivot to user-cuisine profile matrix
    user_cuisine_profiles = cuisine_preferences.pivot_table(
        index='customer_id',
        columns='cuisine_origin',
        values='normalized_score',
        fill_value=0
    )

    # 2. Price Preference Profiles
    price_preferences = df.groupby('customer_id').agg({
        'total_order_value': ['mean', 'std', 'min', 'max'],
        'num_products': 'mean',  # Using num_products as frequency proxy
        'avg_vendor_rating': 'mean'
    }).reset_index()

    price_preferences.columns = ['customer_id', 'avg_order_value', 'value_std', 'min_value', 'max_value', 'avg_products', 'avg_rating']

    # 3. Combine all features into comprehensive user profiles
    user_profiles = pd.DataFrame(index=user_cuisine_matrix.index)

    # Add cuisine preferences
    user_profiles = user_profiles.merge(
        user_cuisine_profiles,
        left_index=True,
        right_index=True,
        how='left'
    ).fillna(0)

    # Add price preferences
    user_profiles = user_profiles.merge(
        price_preferences[['customer_id', 'avg_order_value', 'value_std', 'avg_products']],
        left_index=True,
        right_on='customer_id',
        how='left'
    ).fillna(0)

    return user_profiles, user_cuisine_profiles, price_preferences

# Create content-based profiles
user_profiles, cuisine_profiles, price_profiles = create_content_based_profiles(df)

print(f"User profiles shape: {user_profiles.shape}")

# C) HYBRID RECOMMENDATION FUNCTION
def generate_recommendations(user_id, als_model, user_profiles, vendor_features, top_n=10):
    """
    Generate hybrid recommendations using both collaborative and content-based filtering
    """

    # 1. Collaborative Filtering Recommendations
    if user_id in user_vendor_matrix.index:
        user_idx = user_vendor_matrix.index.get_loc(user_id)
        # Get ALS recommendations
        cf_scores = als_model.item_factors.dot(als_model.user_factors[user_idx])
        cf_recommendations = pd.DataFrame({
            'vendor_id': user_vendor_matrix.columns,
            'cf_score': cf_scores
        }).nlargest(top_n * 2, 'cf_score')  # Get more for filtering
    else:
        # New user - fallback to popular items
        cf_recommendations = pd.DataFrame({
            'vendor_id': user_vendor_matrix.columns,
            'cf_score': user_vendor_matrix.sum(axis=0)  # Overall popularity
        }).nlargest(top_n * 2, 'cf_score')

    # 2. Content-Based Filtering (if user has profile)
    if user_id in user_profiles['customer_id'].values:
        user_profile = user_profiles[user_profiles['customer_id'] == user_id].iloc[0]

        # Calculate content-based scores based on cuisine preferences
        cb_scores = {}
        for vendor_id in cf_recommendations['vendor_id']:
            if vendor_id in vendor_features.index:
                vendor_cuisine = vendor_features.loc[vendor_id, 'cuisine_origin']
                if vendor_cuisine in user_profile.index:
                    cb_scores[vendor_id] = user_profile[vendor_cuisine]
                else:
                    cb_scores[vendor_id] = 0
            else:
                cb_scores[vendor_id] = 0

        # Combine scores
        cf_recommendations['cb_score'] = cf_recommendations['vendor_id'].map(cb_scores)
        cf_recommendations['hybrid_score'] = (
            0.7 * cf_recommendations['cf_score'] +  # Collaborative weighting
            0.3 * cf_recommendations['cb_score']    # Content-based weighting
        )

        final_recommendations = cf_recommendations.nlargest(top_n, 'hybrid_score')
    else:
        final_recommendations = cf_recommendations.head(top_n)

    return final_recommendations

# Prepare vendor features for content-based filtering
vendor_features = df[['vendor_id', 'cuisine_origin', 'vendor_geohash']].drop_duplicates().set_index('vendor_id')

# Test the recommendation function
sample_user = '2e7276ad3a'
recommendations = generate_recommendations(sample_user, als_model, user_profiles, vendor_features)

print("Top recommendations for user:", sample_user)
print(recommendations[['vendor_id', 'cf_score', 'cb_score', 'hybrid_score']].head())


  0%|          | 0/15 [00:00<?, ?it/s]

User factors shape: (2541, 50)
Vendor factors shape: (3309, 50)
User profiles shape: (2541, 21)
Top recommendations for user: 2e7276ad3a
     vendor_id  cf_score  cb_score  hybrid_score
1650  826ba77a  0.106046  0.369658      0.185129
889   4668b21a  0.100499  0.369658      0.181247
282   15fd5daf  0.068001  0.369658      0.158498
579   2e0122a3  0.067472  0.369658      0.158128
1511  79096d7c  0.066353  0.369658      0.157345


In [ ]:

# =============================================
# MODEL PERSISTENCE FOR PHASE 3
# =============================================

import pickle

def save_models():
    """Save all trained models for Phase 3"""
    models = {
        'als_model': als_model,
        'user_vendor_matrix': user_vendor_matrix,
        'user_cuisine_matrix': user_cuisine_matrix,
        'user_profiles': user_profiles,
        'vendor_features': vendor_features,
        'user_factors': user_factors,
        'vendor_factors': vendor_factors
    }

    with open('phase2_models.pkl', 'wb') as f:
        pickle.dump(models, f)

    print("Models saved successfully!")

# Save models
save_models()

Models saved successfully!


In [ ]:
# =============================================
# RECOMMENDATION VALIDATION & ANALYSIS (FIXED)
# =============================================

def analyze_user_preferences_and_recommendations(user_id, df, recommendations, user_profiles, vendor_features, top_n=10):
    """
    Comprehensive analysis to validate if recommendations make sense
    """

    print("=" * 80)
    print(f"🧐 VALIDATING RECOMMENDATIONS FOR USER: {user_id}")
    print("=" * 80)

    # 1. USER'S ORDER HISTORY ANALYSIS
    print("\n📋 1. USER'S ORDER HISTORY:")
    print("-" * 40)

    user_orders = df[df['customer_id'] == user_id]

    if user_orders.empty:
        print("No order history found for this user!")
        return

    print(f"Total Orders: {user_orders['order_id'].nunique()}")
    print(f"Total Vendors Tried: {user_orders['vendor_id'].nunique()}")

    # Most ordered cuisines
    print(f"\n🍽️  TOP CUISINES ORDERED:")
    cuisine_counts = user_orders['cuisine_origin'].value_counts()
    for cuisine, count in cuisine_counts.head(5).items():
        print(f"   - {cuisine}: {count} orders")

    # Most frequent vendors
    print(f"\n🏪 TOP VENDORS ORDERED FROM:")
    vendor_counts = user_orders['vendor_id'].value_counts()
    for vendor, count in vendor_counts.head(5).items():
        vendor_cuisine = user_orders[user_orders['vendor_id'] == vendor]['cuisine_origin'].iloc[0]
        print(f"   - Vendor {vendor} ({vendor_cuisine}): {count} orders")

    # Spending patterns
    avg_spend = user_orders['total_order_value'].mean()
    print(f"\n💰 AVERAGE ORDER VALUE: ${avg_spend:.2f}")

    # 2. USER'S PREFERENCE PROFILE
    print(f"\n🎯 2. USER'S PREFERENCE PROFILE:")
    print("-" * 40)

    if user_id in user_profiles['customer_id'].values:
        user_profile = user_profiles[user_profiles['customer_id'] == user_id].iloc[0]

        # Get top cuisine preferences - FIXED VERSION
        cuisine_cols = [col for col in user_profile.index
                       if col in df['cuisine_origin'].unique()
                       and pd.api.types.is_numeric_dtype(user_profile[col])]

        if cuisine_cols:
            cuisine_prefs = user_profile[cuisine_cols].astype(float)
            # Filter out zero scores and get top cuisines
            non_zero_cuisines = cuisine_prefs[cuisine_prefs > 0]
            if not non_zero_cuisines.empty:
                top_cuisines = non_zero_cuisines.nlargest(5)

                print("TOP CUISINE PREFERENCES (Normalized Scores):")
                for cuisine, score in top_cuisines.items():
                    print(f"   - {cuisine}: {score:.3f}")
            else:
                print("No strong cuisine preferences detected")
                top_cuisines = pd.Series(dtype=float)
        else:
            print("No cuisine preference data available")
            top_cuisines = pd.Series(dtype=float)

    # 3. ANALYZE RECOMMENDATIONS
    print(f"\n🎯 3. RECOMMENDATION ANALYSIS:")
    print("-" * 40)

    print("TOP RECOMMENDED VENDORS:")
    for i, (_, rec) in enumerate(recommendations.head(top_n).iterrows(), 1):
        vendor_id = rec['vendor_id']
        vendor_info = vendor_features.loc[vendor_id] if vendor_id in vendor_features.index else None

        if vendor_info is not None:
            cuisine = vendor_info['cuisine_origin']
            geohash = vendor_info['vendor_geohash']

            # Check if user has ordered from this vendor before
            ordered_before = vendor_id in user_orders['vendor_id'].values
            same_cuisine_pref = cuisine in top_cuisines.index if 'top_cuisines' in locals() and not top_cuisines.empty else False

            print(f"{i}. Vendor {vendor_id}")
            print(f"   Cuisine: {cuisine}")
            print(f"   Location: {geohash}")
            print(f"   CF Score: {rec['cf_score']:.6f}")
            print(f"   CB Score: {rec['cb_score']:.3f}")
            print(f"   Hybrid Score: {rec['hybrid_score']:.6f}")
            print(f"   Ordered Before: {'✅' if ordered_before else '❌'}")
            print(f"   Matches Top Preferences: {'✅' if same_cuisine_pref else '❌'}")
            print()

    # 4. CHECK FOR DIVERSITY IN RECOMMENDATIONS
    print(f"\n🌍 4. RECOMMENDATION DIVERSITY:")
    print("-" * 40)

    recommended_cuisines = []
    for _, rec in recommendations.head(top_n).iterrows():
        vendor_id = rec['vendor_id']
        if vendor_id in vendor_features.index:
            cuisine = vendor_features.loc[vendor_id, 'cuisine_origin']
            recommended_cuisines.append(cuisine)

    unique_cuisines = set(recommended_cuisines)
    print(f"Unique cuisines in top {top_n} recommendations: {len(unique_cuisines)}")
    print("Recommended cuisines:", list(unique_cuisines))

    # 5. COMPARE WITH USER'S PAST BEHAVIOR
    print(f"\n🔍 5. CONSISTENCY WITH USER HISTORY:")
    print("-" * 40)

    user_cuisines = set(user_orders['cuisine_origin'].unique())
    overlap = user_cuisines.intersection(unique_cuisines)

    print(f"Cuisines user has ordered: {len(user_cuisines)}")
    print(f"Cuisines in recommendations: {len(unique_cuisines)}")
    print(f"Overlap: {len(overlap)} cuisines")
    print(f"New cuisines recommended: {len(unique_cuisines - user_cuisines)}")

    # 6. SPECIFIC RECOMMENDATION INSIGHTS
    print(f"\n💡 6. KEY INSIGHTS:")
    print("-" * 40)

    # Check if recommendations match user's favorite cuisines
    if 'top_cuisines' in locals() and not top_cuisines.empty:
        user_top_cuisine = top_cuisines.index[0]
        rec_top_cuisine = max(set(recommended_cuisines), key=recommended_cuisines.count) if recommended_cuisines else "None"

        print(f"User's favorite cuisine: {user_top_cuisine}")
        print(f"Most recommended cuisine: {rec_top_cuisine}")
        print(f"Match: {'✅' if user_top_cuisine == rec_top_cuisine else '❌ (Potential Issue)'}")

    # Check score distribution
    cf_weight = 0.7
    cb_weight = 0.3

    avg_cf_score = recommendations['cf_score'].head(top_n).mean()
    avg_cb_score = recommendations['cb_score'].head(top_n).mean()

    print(f"\nScore Analysis:")
    print(f"Average CF Score: {avg_cf_score:.6f}")
    print(f"Average CB Score: {avg_cb_score:.3f}")
    print(f"CF Contribution: {avg_cf_score * cf_weight:.6f}")
    print(f"CB Contribution: {avg_cb_score * cb_weight:.3f}")

# Let's first check the data types in user_profiles to understand the issue
print("🔍 CHECKING USER_PROFILES DATA TYPES:")
print(user_profiles.dtypes)

# Also let's see what columns we have
print(f"\nUser profiles columns: {list(user_profiles.columns)}")

# Now run the fixed analysis
analyze_user_preferences_and_recommendations(
    user_id='008ab40ac0',
    df=df,
    recommendations=recommendations,
    user_profiles=user_profiles,
    vendor_features=vendor_features,
    top_n=10
)

# =============================================
# QUICK DIAGNOSTIC TO UNDERSTAND THE ISSUE
# =============================================

print("\n🔧 DIAGNOSTIC ANALYSIS OF YOUR RECOMMENDATIONS:")
print("Current recommendation scores show:")
print("- Very low CF scores (~0.000055)")
print("- Very high CB scores (1.0)")
print("This indicates the content-based filtering is dominating")

# Let's check a specific user's profile
sample_user_id = '008ab40ac0'
if sample_user_id in user_profiles['customer_id'].values:
    user_row = user_profiles[user_profiles['customer_id'] == sample_user_id].iloc[0]

    print(f"\nSample user profile structure:")
    print(f"Number of columns: {len(user_row)}")

    # Check cuisine preference columns specifically
    cuisine_pref_cols = [col for col in user_row.index if col in df['cuisine_origin'].unique()]
    print(f"Cuisine preference columns found: {len(cuisine_pref_cols)}")

    if cuisine_pref_cols:
        print("Sample cuisine preferences:")
        for col in cuisine_pref_cols[:5]:  # Show first 5
            print(f"  {col}: {user_row[col]} (type: {type(user_row[col])})")

# =============================================
# IMMEDIATE FIX FOR WEIGHT IMBALANCE
# =============================================

def generate_balanced_recommendations(user_id, als_model, user_profiles, vendor_features, top_n=10, cf_weight=0.9, cb_weight=0.1):
    """Recommendation function with better balanced weights"""

    if user_id in user_vendor_matrix.index:
        user_idx = user_vendor_matrix.index.get_loc(user_id)
        cf_scores = als_model.item_factors.dot(als_model.user_factors[user_idx])
        cf_recommendations = pd.DataFrame({
            'vendor_id': user_vendor_matrix.columns,
            'cf_score': cf_scores
        }).nlargest(top_n * 3, 'cf_score')  # Get more candidates
    else:
        cf_recommendations = pd.DataFrame({
            'vendor_id': user_vendor_matrix.columns,
            'cf_score': user_vendor_matrix.sum(axis=0)
        }).nlargest(top_n * 3, 'cf_score')

    if user_id in user_profiles['customer_id'].values:
        user_profile = user_profiles[user_profiles['customer_id'] == user_id].iloc[0]

        cb_scores = {}
        for vendor_id in cf_recommendations['vendor_id']:
            if vendor_id in vendor_features.index:
                vendor_cuisine = vendor_features.loc[vendor_id, 'cuisine_origin']
                # Safely get cuisine preference score
                if vendor_cuisine in user_profile.index and pd.api.types.is_numeric_dtype(type(user_profile[vendor_cuisine])):
                    try:
                        cb_scores[vendor_id] = float(user_profile[vendor_cuisine])
                    except (ValueError, TypeError):
                        cb_scores[vendor_id] = 0
                else:
                    cb_scores[vendor_id] = 0
            else:
                cb_scores[vendor_id] = 0

        cf_recommendations['cb_score'] = cf_recommendations['vendor_id'].map(cb_scores)

        # Normalize scores before combining
        max_cf = cf_recommendations['cf_score'].max()
        max_cb = cf_recommendations['cb_score'].max() if cf_recommendations['cb_score'].max() > 0 else 1

        cf_recommendations['cf_normalized'] = cf_recommendations['cf_score'] / max_cf
        cf_recommendations['cb_normalized'] = cf_recommendations['cb_score'] / max_cb

        cf_recommendations['hybrid_score'] = (
            cf_weight * cf_recommendations['cf_normalized'] +
            cb_weight * cf_recommendations['cb_normalized']
        )

        final_recommendations = cf_recommendations.nlargest(top_n, 'hybrid_score')
    else:
        final_recommendations = cf_recommendations.head(top_n)

    return final_recommendations

# Test with better balanced approach
print(f"\n🔄 TESTING WITH BETTER BALANCED WEIGHTS:")
balanced_recs = generate_balanced_recommendations(
    user_id='008ab40ac0',
    als_model=als_model,
    user_profiles=user_profiles,
    vendor_features=vendor_features,
    cf_weight=0.9,  # Much higher weight for CF
    cb_weight=0.1   # Much lower weight for CB
)

print("Balanced Recommendations:")
print(balanced_recs[['vendor_id', 'cf_score', 'cb_score', 'hybrid_score']].head(10))

🔍 CHECKING USER_PROFILES DATA TYPES:
american           float64
asian              float64
chinese            float64
french             float64
german             float64
indian             float64
indonesian         float64
italian            float64
japanese           float64
malaysian          float64
mexican            float64
singaporean        float64
snacks             float64
spanish            float64
thai               float64
turkish            float64
vietnamese         float64
customer_id         object
avg_order_value    float64
value_std          float64
avg_products       float64
dtype: object

User profiles columns: ['american', 'asian', 'chinese', 'french', 'german', 'indian', 'indonesian', 'italian', 'japanese', 'malaysian', 'mexican', 'singaporean', 'snacks', 'spanish', 'thai', 'turkish', 'vietnamese', 'customer_id', 'avg_order_value', 'value_std', 'avg_products']
🧐 VALIDATING RECOMMENDATIONS FOR USER: 008ab40ac0

📋 1. USER'S ORDER HISTORY:
------------------------

### **Overview**

📈 Current Results Assessment:
Grade: B+

✅ Excellent for content-based recommendations

❌ Poor for collaborative filtering

✅ Good for user experience (shows relevant Chinese options)

### Different Selection Strategy

In [ ]:
# =============================================
# BETTER USER SELECTION STRATEGIES
# =============================================

def select_test_users(df, strategy="diverse", n_users=5):
    """
    Select users for testing recommendations based on different strategies
    """

    # Calculate user order counts
    user_order_counts = df.groupby('customer_id').size()

    if strategy == "diverse":
        # Select users with different order frequencies
        return {
            'new_user': user_order_counts[user_order_counts == 1].index[0],      # 1 order
            'medium_user': user_order_counts[user_order_counts.between(2, 5)].index[0],  # 2-5 orders
            'power_user': user_order_counts[user_order_counts > 10].index[0],    # 10+ orders
            'random_user': df['customer_id'].sample(1).iloc[0],                  # Random
            'first_user': df['customer_id'].iloc[0]                              # Your current method
        }

    elif strategy == "power_users":
        # Users with most orders (best for testing CF)
        return user_order_counts.nlargest(n_users).index.tolist()

    elif strategy == "new_users":
        # Users with few orders (tests CB fallback)
        return user_order_counts.nsmallest(n_users).index.tolist()

    elif strategy == "coverage":
        # Users who ordered different cuisines
        user_cuisine_diversity = df.groupby('customer_id')['cuisine_origin'].nunique()
        return user_cuisine_diversity.nlargest(n_users).index.tolist()

# Test with diverse users
test_users = select_test_users(df, strategy="diverse")

print("🧪 TESTING WITH DIVERSE USERS:")
for user_type, user_id in test_users.items():
    user_orders = len(df[df['customer_id'] == user_id])
    user_cuisines = df[df['customer_id'] == user_id]['cuisine_origin'].nunique()

    print(f"{user_type}: {user_id} ({user_orders} orders, {user_cuisines} cuisines)")

    # Generate recommendations for this user
    recommendations = generate_recommendations(user_id, als_model, user_profiles, vendor_features)
    top_cuisine = vendor_features.loc[recommendations.iloc[0]['vendor_id'], 'cuisine_origin']

    print(f"  Top recommendation: {top_cuisine}")
    print(f"  CF score: {recommendations.iloc[0]['cf_score']:.6f}")
    print(f"  CB score: {recommendations.iloc[0]['cb_score']:.3f}")
    print()

🧪 TESTING WITH DIVERSE USERS:
new_user: 008ab40ac0 (1 orders, 1 cuisines)
  Top recommendation: chinese
  CF score: 0.000055
  CB score: 1.000

medium_user: 008ce71183 (3 orders, 2 cuisines)
  Top recommendation: american
  CF score: 0.045964
  CB score: 0.679

power_user: 046958a898 (12 orders, 4 cuisines)
  Top recommendation: snacks
  CF score: 0.039190
  CB score: 0.487

random_user: 046958a898 (12 orders, 4 cuisines)
  Top recommendation: snacks
  CF score: 0.039190
  CB score: 0.487

first_user: 008ab40ac0 (1 orders, 1 cuisines)
  Top recommendation: chinese
  CF score: 0.000055
  CB score: 1.000



In [13]:
# Let's investigate the power user more closely
power_user_id = '046958a898'
power_user_orders = df[df['customer_id'] == power_user_id]

print(f"🔍 ANALYZING POWER USER: {power_user_id}")
print("=" * 50)

# Check their actual order history
print("📋 ORDER HISTORY:")
cuisine_history = power_user_orders['cuisine_origin'].value_counts()
for cuisine, count in cuisine_history.items():
    print(f"  - {cuisine}: {count} orders")

# Check their preference profile
if power_user_id in user_profiles['customer_id'].values:
    user_profile = user_profiles[user_profiles['customer_id'] == power_user_id].iloc[0]
    print(f"\n🎯 PREFERENCE PROFILE:")
    # Get top 3 cuisine preferences
    cuisine_cols = [col for col in user_profile.index if col in df['cuisine_origin'].unique()]
    cuisine_prefs = user_profile[cuisine_cols].astype(float)
    top_preferences = cuisine_prefs.nlargest(3)
    for cuisine, score in top_preferences.items():
        print(f"  - {cuisine}: {score:.3f}")

# Check why snacks was recommended
snacks_recommendation = recommendations[recommendations['vendor_id'] == recommendations.iloc[0]['vendor_id']]
print(f"\n🤔 WHY SNACKS RECOMMENDATION?")
print(f"Vendor: {snacks_recommendation['vendor_id'].iloc[0]}")
print(f"CF Score: {snacks_recommendation['cf_score'].iloc[0]:.6f}")
print(f"CB Score: {snacks_recommendation['cb_score'].iloc[0]:.3f}")

🔍 ANALYZING POWER USER: 046958a898
📋 ORDER HISTORY:
  - snacks: 6 orders
  - american: 3 orders
  - italian: 2 orders
  - singaporean: 1 orders

🎯 PREFERENCE PROFILE:
  - snacks: 0.487
  - american: 0.249
  - italian: 0.188

🤔 WHY SNACKS RECOMMENDATION?
Vendor: 90cca11f
CF Score: 0.000055
CB Score: 1.000


In [ ]:
# =============================================
# COMPREHENSIVE MODEL EVALUATION
# =============================================

def evaluate_model_performance(df, als_model, user_profiles, vendor_features, n_test_users=10):
    """
    Evaluate recommendation quality across different user types
    """

    user_order_counts = df.groupby('customer_id').size()

    print("📊 MODEL PERFORMANCE EVALUATION")
    print("=" * 50)

    # Test different user segments
    segments = {
        'New Users (1 order)': user_order_counts[user_order_counts == 1].index[:3],
        'Medium Users (2-5 orders)': user_order_counts[user_order_counts.between(2, 5)].index[:3],
        'Power Users (6+ orders)': user_order_counts[user_order_counts >= 6].index[:3],
        'Diverse Cuisine Users': df.groupby('customer_id')['cuisine_origin'].nunique().nlargest(3).index
    }

    for segment_name, user_ids in segments.items():
        print(f"\n🎯 {segment_name}:")
        print("-" * 30)

        for user_id in user_ids:
            if user_id in user_vendor_matrix.index:
                recommendations = generate_recommendations(user_id, als_model, user_profiles, vendor_features)

                # Basic user stats
                user_orders = len(df[df['customer_id'] == user_id])
                user_cuisines = df[df['customer_id'] == user_id]['cuisine_origin'].nunique()

                # Recommendation stats
                top_rec = recommendations.iloc[0]
                top_cuisine = vendor_features.loc[top_rec['vendor_id'], 'cuisine_origin']

                print(f"  User {user_id[:8]}...: {user_orders} orders, {user_cuisines} cuisines")
                print(f"    → Top rec: {top_cuisine} (CF: {top_rec['cf_score']:.4f}, CB: {top_rec['cb_score']:.2f})")

# Run comprehensive evaluation
evaluate_model_performance(df, als_model, user_profiles, vendor_features)

📊 MODEL PERFORMANCE EVALUATION

🎯 New Users (1 order):
------------------------------
  User 008ab40a...: 1 orders, 1 cuisines
    → Top rec: chinese (CF: 0.0001, CB: 1.00)
  User 00ba08ba...: 1 orders, 1 cuisines
    → Top rec: american (CF: 0.0342, CB: 1.00)
  User 00e9c13b...: 1 orders, 1 cuisines
    → Top rec: american (CF: 0.0098, CB: 1.00)

🎯 Medium Users (2-5 orders):
------------------------------
  User 008ce711...: 3 orders, 2 cuisines
    → Top rec: american (CF: 0.0460, CB: 0.68)
  User 00c41737...: 2 orders, 2 cuisines
    → Top rec: chinese (CF: 0.0267, CB: 0.49)
  User 01058d1d...: 2 orders, 1 cuisines
    → Top rec: snacks (CF: 0.1304, CB: 1.00)

🎯 Power Users (6+ orders):
------------------------------
  User 01907d62...: 9 orders, 2 cuisines
    → Top rec: snacks (CF: 0.0977, CB: 0.65)
  User 027862ef...: 6 orders, 3 cuisines
    → Top rec: chinese (CF: 0.0216, CB: 0.66)
  User 028a29c2...: 6 orders, 2 cuisines
    → Top rec: snacks (CF: 0.0577, CB: 0.48)

🎯 Diverse 